# Initial Set-Up

In [4]:
import pickle
import pandas as pd
import matplotlib
import os
import re

import collections
import datetime
import time

import geopandas as gpd

import numpy as np

import rasterio

from difflib import get_close_matches

from fuzzywuzzy import process
from fuzzywuzzy import fuzz

# Infogroup
This is where we find the weights by company location.

The code commented out below lets us scale if we do the full infogroup file.

In [28]:
import dask.dataframe as dd
df = dd.read_csv( "../../data/companyData/allIGData.csv", assume_missing=True, 
                 dtype={'parent_number': 'object','parent_employee_size_code': 'object',
                       'parent_sales_volume_code': 'object',
                       'abi': 'object'})[['archive_version_year', 'parent_number',
                       'employee_size_location','latitude', 'longitude']]

In [30]:
df = df[(~df.parent_number.isna()) & (~df.employee_size_location.isna())]

In [36]:
employeesByParents = pd.read_csv('../../data/companyData/totalEmployees.csv',
                                dtype={'parent_number': 'object'}).drop(columns = {'Unnamed: 0'})
employeesByParents.head()

,archive_version_year,parent_number,allEmployees
0,2003.0,000007609,0.0
1,2003.0,000023077,68.0
2,2003.0,000076547,287.0
3,2003.0,000077743,649.0
4,2003.0,000089151,188.0


In [37]:
toCalculate = df.merge(employeesByParents)

In [38]:
toCalculate.head()

/Users/brianreed/opt/anaconda3/lib/python3.7/site-packages/dask/utils.py:29: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  return func(*args, **kwargs)


,archive_version_year,parent_number,employee_size_location,latitude,longitude,allEmployees
0,2003.0,007508146,125.0,33.24433,-87.49823,4731.0
1,2003.0,007508146,30.0,33.86970,-117.53745,4731.0
2,2003.0,007508146,170.0,40.51728,-75.55581,4731.0
3,2003.0,007508146,400.0,42.09046,-76.78435,4731.0
4,2003.0,007508146,15.0,33.47081,-86.76147,4731.0


In [40]:
toCalculate['locationFracOfEmployees'] = toCalculate['employee_size_location']/toCalculate['allEmployees']

In [54]:
fractions = toCalculate[['archive_version_year','parent_number','latitude','longitude','locationFracOfEmployees']].\
    groupby(['archive_version_year','parent_number','latitude','longitude']).sum().compute(workers = 100)

/Users/brianreed/opt/anaconda3/lib/python3.7/site-packages/dask/utils.py:29: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  return func(*args, **kwargs)
/Users/brianreed/opt/anaconda3/lib/python3.7/site-packages/dask/utils.py:29: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  return func(*args, **kwargs)
/Users/brianreed/opt/anaconda3/lib/python3.7/site-packages/dask/utils.py:29: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  return func(*args, **kwargs)
/Users/brianreed/opt/anaconda3/lib/python3.7/site-packages/dask/utils.py:29: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  return func(*args, **kwargs)
/Users/brianreed/opt/anaconda3/lib/python3.7/site-packages/dask/utils.py:29: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=Fal

/Users/brianreed/opt/anaconda3/lib/python3.7/site-packages/dask/utils.py:29: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  return func(*args, **kwargs)
/Users/brianreed/opt/anaconda3/lib/python3.7/site-packages/dask/utils.py:29: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  return func(*args, **kwargs)
/Users/brianreed/opt/anaconda3/lib/python3.7/site-packages/dask/utils.py:29: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  return func(*args, **kwargs)
/Users/brianreed/opt/anaconda3/lib/python3.7/site-packages/dask/utils.py:29: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  return func(*args, **kwargs)
/Users/brianreed/opt/anaconda3/lib/python3.7/site-packages/dask/utils.py:29: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=Fal

In [55]:
fractions = fractions.reset_index()
fractions.head()

,archive_version_year,parent_number,latitude,longitude,locationFracOfEmployees
0,2003.0,000023077,40.47235,-80.06152,0.735294
1,2003.0,000076547,26.55504,-81.88423,0.121951
2,2003.0,000077743,33.86604,-117.89996,0.078582
3,2003.0,000077743,34.15562,-118.65163,0.018490
4,2003.0,000089151,38.88735,-84.80038,0.265957


In [57]:
fractions.to_csv('../../data/companyData/fractionEmployees_byEstablishment.csv')

In [46]:
gvKey_abiLinkingTable = pd.read_csv('../../data/companyData/linkingTable.csv')

In [45]:
fractions.to_csv('../../data/companyData/fractionEmployees.csv')

KeyboardInterrupt: 

In [7]:
df2010s = df[df.archive_version_year > 2007]

In [7]:
df.head()

/Users/brianreed/opt/anaconda3/lib/python3.7/site-packages/dask/utils.py:29: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  return func(*args, **kwargs)


,archive_version_year,abi,ticker,parent_number,company,address_line_1,city,state,zipcode,zip4,...,location_sales_volume_code,primary_naics_code,sic_code,business_status_code,company_holding_status,cbsa_code,employee_size_location,sales_volume_location,latitude,longitude
0,2003.0,000000737,NaN,007508146,EMPIRE COKE CO,3200 MAIN ST NE,HOLT,AL,35404.0,1100.0,...,H,33111009.0,505201.0,2.0,NaN,46220.0,125.0,63500.0,33.24433,-87.49823
1,2003.0,000001099,NaN,NaN,ELDER MANUFACTURING,720 N 3RD AVE,PARAGOULD,AR,72450.0,3077.0,...,D,31522009.0,565101.0,9.0,NaN,37500.0,32.0,4096.0,36.06316,-90.48386
2,2003.0,000001255,NaN,NaN,INDIANA STEEL & WIRE CO,2200 E JACKSON ST,MUNCIE,IN,47303.0,4141.0,...,G,33122205.0,331601.0,9.0,NaN,34620.0,290.0,45000.0,40.19310,-85.36173
3,2003.0,000002006,NaN,441297108,U S FOODSERVICE,7950 SPENCE RD,FAIRBURN,GA,30213.0,2953.0,...,I,42441005.0,514404.0,2.0,NaN,12060.0,168.0,191016.0,33.54118,-84.55948
4,2003.0,000002055,NaN,NaN,GOLDEN-ROD BROILER INC,2352 COUNTY ROAD 719,CULLMAN,AL,35055.0,9655.0,...,H,31161501.0,514404.0,9.0,NaN,18980.0,500.0,93000.0,34.15419,-86.76856


In [8]:
df.columns

Index(['archive_version_year', 'abi', 'ticker', 'parent_number', 'company',
       'address_line_1', 'city', 'state', 'zipcode', 'zip4', 'idcode',
       'location_employee_size_code', 'location_sales_volume_code',
       'primary_naics_code', 'sic_code', 'business_status_code',
       'company_holding_status', 'cbsa_code', 'employee_size_location',
       'sales_volume_location', 'latitude', 'longitude'],
      dtype='object')

In [ ]:
hq = hq[(hq.business_status_code == 1.0)]

In [ ]:
hq.shape

In [ ]:
hq.head()

In [ ]:
hq.to_csv('../../data/companyData/hqPublicAll.csv')

In [ ]:
hq.head()

# Create df

Convert into geospatial dataframe.

In [ ]:
geoHQ = gpd.GeoDataFrame(
    hq,
    geometry=gpd.points_from_xy(
        hq["longitude"],
        hq["latitude"],
    ),
    crs={"init":"EPSG:4326"},
)

In [ ]:
geoHQ[geoHQ.archive_version_year == 1997].plot()

This is the most basic version of the HQ information. BS takes it a step further by filtering for firms where the HQ are responsible for at least a certain percentage of the overall sales.

In [7]:
os.getcwd()

'/Users/brianreed/Documents/supplyChain/extremes/extremesCleaningCode'

In [8]:
file = "../../data/companyData/infogroup.csv"
hqSales = pd.read_csv(file, encoding = 'unicode_escape')
hqSales = hqSales[hqSales.business_status_code == 1.0]

# hqSales = hq

FileNotFoundError: [Errno 2] No such file or directory: '../../data/companyData/infogroup.csv'

Overstate the size of a location relative to the parent: put parent at low end and location at high end of ranges.

In [ ]:
hqSales.loc[hqSales.parent_employee_size_code == 'A', 'parent_employee_size_code'] = 1
hqSales.loc[hqSales.parent_employee_size_code == 'B', 'parent_employee_size_code'] = 5
hqSales.loc[hqSales.parent_employee_size_code == 'C', 'parent_employee_size_code'] = 10
hqSales.loc[hqSales.parent_employee_size_code == 'D', 'parent_employee_size_code'] = 20
hqSales.loc[hqSales.parent_employee_size_code == 'E', 'parent_employee_size_code'] = 50
hqSales.loc[hqSales.parent_employee_size_code == 'F', 'parent_employee_size_code'] = 100
hqSales.loc[hqSales.parent_employee_size_code == 'G', 'parent_employee_size_code'] = 250
hqSales.loc[hqSales.parent_employee_size_code == 'H', 'parent_employee_size_code'] = 500
hqSales.loc[hqSales.parent_employee_size_code == 'I', 'parent_employee_size_code'] = 1000
hqSales.loc[hqSales.parent_employee_size_code == 'J', 'parent_employee_size_code'] = 5000
hqSales.loc[hqSales.parent_employee_size_code == 'K', 'parent_employee_size_code'] = 10000


hqSales.loc[hqSales.location_employee_size_code == 'A', 'location_employee_size_code'] = 4
hqSales.loc[hqSales.location_employee_size_code == 'B', 'location_employee_size_code'] = 9
hqSales.loc[hqSales.location_employee_size_code == 'C', 'location_employee_size_code'] = 19
hqSales.loc[hqSales.location_employee_size_code == 'D', 'location_employee_size_code'] = 49
hqSales.loc[hqSales.location_employee_size_code == 'E', 'location_employee_size_code'] = 99
hqSales.loc[hqSales.location_employee_size_code == 'F', 'location_employee_size_code'] = 249
hqSales.loc[hqSales.location_employee_size_code == 'G', 'location_employee_size_code'] = 499
hqSales.loc[hqSales.location_employee_size_code == 'H', 'location_employee_size_code'] = 999
hqSales.loc[hqSales.location_employee_size_code == 'I', 'location_employee_size_code'] = 4999
hqSales.loc[hqSales.location_employee_size_code == 'J', 'location_employee_size_code'] = 9999
hqSales.loc[hqSales.location_employee_size_code == 'K', 'location_employee_size_code'] = 10000


hqSales['location_employee_size_code'] = pd.to_numeric(hqSales['location_employee_size_code'], errors='coerce')
hqSales['parent_employee_size_code'] = pd.to_numeric(hqSales['parent_employee_size_code'], errors='coerce')
hqSales['employeesAtLocation'] = hqSales['location_employee_size_code']/hqSales['parent_employee_size_code']




In [ ]:
hqSales['employeesAtLocation'].describe()

In [ ]:
sum(hqSales.employeesAtLocation > 0.75)/hqSales.shape[0]

Filter down the location information to firms where we have at least 75% of employees at HQ.

In [ ]:
geoHQ = geoHQ[hqSales.employeesAtLocation > 0.75].reset_index()

In [ ]:
geoHQ.drop(columns = ['index','geometry']).to_csv('data/geoHQ.csv')

In [ ]:
geoHQ = pd.read_csv("../../data/geoHQ.csv")

In [ ]:
geoHQ.shape

In [ ]:
geoHQ.head()

In [ ]:
compustat1 = pd.read_csv("../../data/companyData/compustatChanges_All.csv").drop(columns = ['Unnamed: 0'])

compustat2 = pd.read_csv("../../data/companyData/compustatChanges_2010s.csv").drop(columns = ['Unnamed: 0','naics'])

compustat = compustat1.append(compustat2)

compustat.rename(columns = {'tic': 'TICKER'}, inplace = True)

print(compustat1.columns,compustat2.columns)

In [ ]:
print(compustat.shape, compustat.head())

In [ ]:
cstatCos = compustat[['year','companyName','TICKER']].drop_duplicates()
cstatCos = cstatCos[cstatCos.year > 1996]

geoHQ.rename(columns = {'archive_version_year': 'year'},inplace = True)

In [ ]:
geoHQ.head()

In [ ]:
allIG = cstatCos.merge(geoHQ)
allIG.to_csv("../../data/igHQ.csv")

# Weather Data
## *this is how we process the cmip6Data


In [ ]:
import netCDF4 as nc
import xarray as xr

In [ ]:
file = '../../../../../../../Volumes/backup2/dissData/cmip6Data/hist/pr/pr_day_MIROC6_historical_r7i1p1f1_gn_20100101-20141231.nc'
data = xr.open_dataset(file)   #  nc.Dataset(file)

In [ ]:
data

In [ ]:
geoHQ = gpd.GeoDataFrame(
    allIG,
    geometry=gpd.points_from_xy(
        allIG["longitude"],
        allIG["latitude"],
    ),
    crs={"init":"EPSG:4326"},
)

In [ ]:
geoHQ['longitude']

In [ ]:
one_point = data["pr"].sel(lat=50, lon=50)

In [ ]:
pd.DataFrame(test.temperature[test.year == 2010]).plot()

Now go through and make this iterable for the entire list.

In [ ]:
i = 0
data.pr.sel(allIG.iloc[0,:], method='nearest')

In [ ]:
hqPost2010 = geoHQ[geoHQ.archive_version_year > 2010].reset_index()

In [ ]:
start = time.time()



tempsList = list()
for i in range(0,hqPost2010.shape[0]):
    nearbyTemps = temps.sel(hqPost2010.loc[i,['latitude','longitude']], method='nearest')
    
    tempsList.append(list(np.array(nearbyTemps.temperature[nearbyTemps.year == hqPost2010.archive_version_year[i]])))

    if (i%100 == 0):
        print(i)
    
print(time.time() - start)






In [ ]:
outfile =  'data/hqDailyTemperatures.pkl'
with open(outfile, 'wb') as pickle_file:
    pickle.dump(tempsList, pickle_file)